In [ ]:
def create_model(model_name, n_points=0):

        if model_name == "SIGMOID_HEATMAP_SIGMOID_REGRESS_TWO_HEAD":
            return BlazePoseLegacy(n_points).build_model("TWO_HEAD")
        elif model_name == "SIGMOID_HEATMAP_SIGMOID_REGRESS_HEATMAP":
            return BlazePoseLegacy(n_points).build_model("HEATMAP")
        elif model_name == "SIGMOID_HEATMAP_SIGMOID_REGRESS_REGRESSION":
            return BlazePoseLegacy(n_points).build_model("REGRESSION")

        elif model_name == "SIGMOID_HEATMAP_LINEAR_REGRESS_TWO_HEAD":
            return BlazePose(n_points).build_model("TWO_HEAD")
        elif model_name == "SIGMOID_HEATMAP_LINEAR_REGRESS_HEATMAP":
            return BlazePose(n_points).build_model("HEATMAP")
        elif model_name == "SIGMOID_HEATMAP_LINEAR_REGRESS_REGRESSION":
            return BlazePose(n_points).build_model("REGRESSION")

        elif model_name == "ALL_LINEAR_TWO_HEAD":
            return BlazePoseAllLinear(n_points).build_model("TWO_HEAD")
        elif model_name == "ALL_LINEAR_HEATMAP":
            return BlazePoseAllLinear(n_points).build_model("HEATMAP")
        elif model_name == "ALL_LINEAR_REGRESSION":
            return BlazePoseAllLinear(n_points).build_model("REGRESSION")

        elif model_namModelCreator.e == "PUSHUP_RECOGNITION":
            return PushUpRecognition.build_model()

        elif model_name == "BLAZEPOSE_WITH_PUSHUP_CLASSIFY":
            return BlazePoseWithClassify(n_points).build_model("TWO_HEAD")

In [ ]:
class ChannelPadding(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(ChannelPadding, self).__init__()
        self.channels = channels

    def build(self, input_shapes):
        self.pad_shape = tf.constant(
            [[0, 0], [0, 0], [0, 0], [0, self.channels - input_shapes[-1]]])

    def call(self, x):
        return tf.pad(x, self.pad_shape)


class BlazeBlock(tf.keras.Model):
    def __init__(self, block_num=3, channel=48, channel_padding=1, name_prefix=""):
        super(BlazeBlock, self).__init__()

        self.downsample_a = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(kernel_size=3, strides=(
                2, 2), padding='same', activation=None, name=name_prefix+"downsample_a_depthwise"),
            tf.keras.layers.Conv2D(
                filters=channel, kernel_size=1, activation=None, name=name_prefix+"downsample_a_conv1x1")
        ])
        if channel_padding:
            self.downsample_b = tf.keras.models.Sequential([
                tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
                ChannelPadding(channels=channel)
            ])
        else:
            self.downsample_b = tf.keras.layers.MaxPool2D(pool_size=(2, 2))

        self.conv = list()
        for i in range(block_num):
            self.conv.append(tf.keras.models.Sequential([
                tf.keras.layers.DepthwiseConv2D(
                    kernel_size=3, padding='same', activation=None, name=name_prefix+"conv_block_{}".format(i+1)),
                tf.keras.layers.Conv2D(
                    filters=channel, kernel_size=1, activation=None)
            ]))

    def call(self, x):
        x = tf.keras.activations.relu(
            self.downsample_a(x) + self.downsample_b(x))
        for i in range(len(self.conv)):
            x = tf.keras.activations.relu(x + self.conv[i](x))
        return x

In [ ]:
class BlazePose():
    def __init__(self, num_keypoints: int):

        self.num_keypoints = num_keypoints

        self.conv1 = tf.keras.layers.Conv2D(
            filters=24, kernel_size=3, strides=(2, 2), padding='same', activation='relu'
        )

        self.conv2_1 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding='same', activation=None),
            tf.keras.layers.Conv2D(filters=24, kernel_size=1, activation=None)
        ])

        self.conv2_2 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding='same', activation=None),
            tf.keras.layers.Conv2D(filters=24, kernel_size=1, activation=None)
        ])

        # === Heatmap ===

        self.conv3 = BlazeBlock(block_num=3, channel=48)
        self.conv4 = BlazeBlock(block_num=4, channel=96)
        self.conv5 = BlazeBlock(block_num=5, channel=192)
        self.conv6 = BlazeBlock(block_num=6, channel=288)

        self.conv7a = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=48, kernel_size=1, activation="relu"),
            tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="bilinear")
        ])
        self.conv7b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=48, kernel_size=1, activation="relu")
        ])

        self.conv8a = tf.keras.layers.UpSampling2D(
            size=(2, 2), interpolation="bilinear")
        self.conv8b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=48, kernel_size=1, activation="relu")
        ])

        self.conv9a = tf.keras.layers.UpSampling2D(
            size=(2, 2), interpolation="bilinear")
        self.conv9b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=48, kernel_size=1, activation="relu")
        ])

        self.conv10a = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=8, kernel_size=1, activation="relu"),
            tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="bilinear")
        ])
        self.conv10b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(filters=8, kernel_size=1, activation="relu")
        ])

        self.conv11 = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None),
            tf.keras.layers.Conv2D(
                filters=8, kernel_size=1, activation="relu"),
            tf.keras.layers.Conv2D(
                filters=self.num_keypoints, kernel_size=3, padding="same", activation=None) # -> Heatmap output
        ])

        # === Regression ===

        #  In: 1, 64, 64, 48)
        self.conv12a = BlazeBlock(block_num=4, channel=96, name_prefix="regression_conv12a_")    # input res: 64
        self.conv12b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None, name="regression_conv12b_depthwise"),
            tf.keras.layers.Conv2D(
                filters=96, kernel_size=1, activation="relu", name="regression_conv12b_conv1x1")
        ], name="regression_conv12b")

        self.conv13a = BlazeBlock(block_num=5, channel=192, name_prefix="regression_conv13a_")   # input res: 32
        self.conv13b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None, name="regression_conv13b_depthwise"),
            tf.keras.layers.Conv2D(
                filters=192, kernel_size=1, activation="relu", name="regression_conv13b_conv1x1")
        ], name="regression_conv13b")

        self.conv14a = BlazeBlock(block_num=6, channel=288, name_prefix="regression_conv14a_")   # input res: 16
        self.conv14b = tf.keras.models.Sequential([
            tf.keras.layers.DepthwiseConv2D(
                kernel_size=3, padding="same", activation=None, name="regression_conv14b_depthwise"),
            tf.keras.layers.Conv2D(
                filters=288, kernel_size=1, activation="relu", name="regression_conv14b_conv1x1")
        ], name="regression_conv14b")

        self.conv15 = tf.keras.models.Sequential([
            BlazeBlock(block_num=7, channel=288, channel_padding=0, name_prefix="regression_conv15a_"),
            BlazeBlock(block_num=7, channel=288, channel_padding=0, name_prefix="regression_conv15b_")
        ], name="regression_conv15")

        self.conv16 = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(
                filters=3*self.num_keypoints, kernel_size=2, activation=None),
            tf.keras.layers.Reshape((3*self.num_keypoints, 1), name="regression_final_dense")
        ], name="joints")

    def build_model(self, model_type):

        input_x = tf.keras.layers.Input(shape=(256, 256, 3))

        # Block 1
        # In: 1x256x256x3
        x = self.conv1(input_x)

        # Block 2
        # In: 1x128x128x24
        x = x + self.conv2_1(x)
        x = tf.keras.activations.relu(x)

        # Block 3
        # In: 1x128x128x24
        x = x + self.conv2_2(x)
        y0 = tf.keras.activations.relu(x)

        # === Heatmap ===

        # In: 1, 128, 128, 24
        y1 = self.conv3(y0)
        y2 = self.conv4(y1)
        y3 = self.conv5(y2)
        y4 = self.conv6(y3)

        x = self.conv7a(y4) + self.conv7b(y3)
        x = self.conv8a(x) + self.conv8b(y2)
        # In: 1, 32, 32, 96
        x = self.conv9a(x) + self.conv9b(y1)
        # In: 1, 64, 64, 48
        y = self.conv10a(x) + self.conv10b(y0)
        y = self.conv11(y)

        # In: 1, 128, 128, 8
        heatmap = tf.keras.layers.Activation("sigmoid", name="heatmap")(y)

        # === Regression ===

        # Stop gradient for regression on 2-head model
        if model_type == "TWO_HEAD":
            x = tf.keras.backend.stop_gradient(x)
            y2 = tf.keras.backend.stop_gradient(y2)
            y3 = tf.keras.backend.stop_gradient(y3)
            y4 = tf.keras.backend.stop_gradient(y4)

        x = self.conv12a(x) + self.conv12b(y2)
        # In: 1, 32, 32, 96
        x = self.conv13a(x) + self.conv13b(y3)
        # In: 1, 16, 16, 192
        x = self.conv14a(x) + self.conv14b(y4)
        # In: 1, 8, 8, 288
        x = self.conv15(x)
        # In: 1, 2, 2, 288
        joints = self.conv16(x)

        if model_type == "TWO_HEAD":
            return Model(inputs=input_x, outputs=[joints, heatmap])
        elif model_type == "HEATMAP":
            return Model(inputs=input_x, outputs=heatmap)
        elif model_type == "REGRESSION":
            return Model(inputs=input_x, outputs=joints)
        else:
            raise ValueError("Wrong model type.")


In [62]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import json
from tensorflow.keras.models import Model
def set_env():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Restrict TensorFlow to only use the fourth GPU
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

def runnmodel(config, model_path):
    """Load pretrained model

    Args:
        config (dict): Model configuration
        model (str): Path to h5 model to be tested
    """

    model_config = config["model"]

    # Initialize model and load weights
    model =create_model(model_config["model_type"])
    model.compile()
    #model.load_weights(model_path)

    return model


#def return_model(filepath):

  #  model = load_model(filepath)
  #  model.compile()
   # return model

def get_preds(model , image):
    img = np.expand_dims(image , axis = 0)
    print(img )
    preds = model.predict(img)
    print(preds)
    return preds

def convert_preds_to_xy(preds):

    kpts = []
    #temp = preds[2][0]
    temp = preds[1][0]
    for x,y in zip(temp[::4] , temp[1::4]):
        kpts.append(((x),(y)))
        #kpts.append((int(x),int(y)))
      
    return kpts 

def infer_video(model , video = 0 ):
    cap = cv2.VideoCapture(video)

    while cap.isOpened():
        okay , frame = cap.read()
        if not okay :
            print('Cant open webcam , please try again!')
            break

        
        inframe = frame.copy()
        inframe_resize = cv2.resize(inframe , (256 , 256)) / 255
        
        preds = get_preds(model , inframe_resize)
        kpts = convert_preds_to_xy(preds)

        for pair in POSE_PAIRS:
            cv2.line(inframe_resize, kpts[pair[0]], kpts[pair[1]], (0, 255, 0), thickness=1)
        # for point in kpts:
        #     cv2.circle(inframe_resize , point , 2, (0,0,255) , 2)
        cv2.imshow('Inference' , inframe_resize)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def infer_image(model , image):
    img = cv2.imread(image)
    img_resize = cv2.resize(img , (256 , 256)) / 255
    preds = get_preds(model , img_resize)

    kpts = convert_preds_to_xy(preds)
    print(len(kpts))
    for pair in POSE_PAIRS:
        cv2.line(img_resize, kpts[pair[0]], kpts[pair[1]], (0, 255, 0), thickness=1)
    #error: only size-1 arrays can be converted to Python scalars

    # for idx , point in enumerate(kpts):
    #     cv2.circle(img_resize , point, 2 , (0 , 0 , 255) , 2)
    #     cv2.putText(img_resize, "{}".format(idx), point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0 , 255 ,0), 1, lineType=cv2.LINE_AA)   
        # cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), thickness=line_thickness)
    cv2.imshow('Inference Image' , img_resize)
    cv2.waitKey(0)



#if __name__ == '__main__':


    

In [ ]:
POSE_PAIRS = [(11 ,12) , (12 ,14) ,(11,13) ,(13,15) ,(14,16) ,(0,11),(0,12)
                ]
set_env()

with open('config.json') as config_buffer:
    config = json.loads(config_buffer.read())
    
model = runnmodel(config,'path to h5 model')
#model.summary()
#infer_video(model)
infer_image(model , 'image.jpg')